# Noise Acquistion

In this notebook we shall explore how to acquire real interferometer output data, and use that data as a TensorFlow dataset. In GravyFlow, datasets are built by composition, so by combining different elements i.e, noise, injections, conditioning, we can build custom datasets to suit our specifications.

First we shall start by performing the neccisary imports.

In [25]:
# Built-in imports
import os
import sys
from typing import Iterator, List
from pathlib import Path
from itertools import islice

# Dependancy imports: 
import tensorflow as tf
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot

# GravyFlow import, again adding the grandparent directory to the path:

# Get the absolute path of the parent directory
parent_dir = os.path.abspath('../../')
print(parent_dir)

# Add the parent directory to sys.path
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# To import gravyflow simply use:
import gravyflow as gf

/home/michael.norman/data_ad_infinitum


In [26]:
gf.Defaults.set(
    sample_rate_hertz=2048.0,
    onsource_duration_seconds=1.0,
    offsource_duration_seconds=16.0,
    crop_duration_seconds=0.5,
    scale_factor=1.0E21
)

In [3]:
# Setup ifo data acquisition object:
ifo_data_obtainer : gf.IFODataObtainer = gf.IFODataObtainer(
    gf.ObservingRun.O3, 
    gf.DataQuality.BEST, 
    [
        gf.DataLabel.NOISE, 
        gf.DataLabel.GLITCHES
    ],
    gf.SegmentOrder.RANDOM,
    force_acquisition = True,
    cache_segments = False
)

In [4]:
# Initilise noise generator wrapper:
noise : gf.NoiseObtainer = gf.NoiseObtainer(
    ifo_data_obtainer = ifo_data_obtainer,
    noise_type = gf.NoiseType.REAL,
    ifos = gf.IFO.L1
)

In [9]:
onsource, offsource, gps_times = next(noise(num_examples_per_batch=1))

SyntaxError: invalid syntax (3191022957.py, line 1)

In [8]:
 onsource_strain_plot = gf.generate_strain_plot(
        {"Onsource Noise" : onsource[0]},
        title = f"Onsource Background noise at {gps_times[0]}"
    )
    
offsource_strain_plot = gf.generate_strain_plot(
        {"Offsource Noise" : offsource[0]},
        title = f"Offsource Background noise at {gps_times[0]}"
    )

layout : List = [[onsource_strain_plot, offsource_strain_plot]]

# Arrange the plots in a grid. 
grid = gridplot(layout)
output_notebook()
show(grid)

Loading BokehJS ...

In [19]:
num_iterations : int = 16

for onsource, offsource, gps_times in islice(noise(), num_iterations):
    print(f"Got {onsource.shape[0]} more noise Examples! Should probably do something usefull with them!")

INFO:root:The file generator_data/segment_data_365d2f598e3e984cfd47499ae15209bdb76436dd.hdf5 was opened in r mode.


Got 32 more noise Examples! Should probably do something usefull with them!
Got 32 more noise Examples! Should probably do something usefull with them!
Got 32 more noise Examples! Should probably do something usefull with them!
Got 32 more noise Examples! Should probably do something usefull with them!
Got 32 more noise Examples! Should probably do something usefull with them!
Got 32 more noise Examples! Should probably do something usefull with them!
Got 32 more noise Examples! Should probably do something usefull with them!
Got 32 more noise Examples! Should probably do something usefull with them!
Got 32 more noise Examples! Should probably do something usefull with them!
Got 32 more noise Examples! Should probably do something usefull with them!
Got 32 more noise Examples! Should probably do something usefull with them!
Got 32 more noise Examples! Should probably do something usefull with them!
Got 32 more noise Examples! Should probably do something usefull with them!
Got 32 more 

In [20]:
# Initilise noise generator wrapper:
multi_ifo_noise : gf.NoiseObtainer = gf.NoiseObtainer(
        ifo_data_obtainer = ifo_data_obtainer,
        noise_type = gf.NoiseType.REAL,
        ifos = [gf.IFO.L1, gf.IFO.H1]
    )

In [22]:
multi_onsource, multi_offsource, multi_gps_times = next(multi_ifo_noise(num_examples_per_batch=1))

INFO:root:The file generator_data/segment_data_365d2f598e3e984cfd47499ae15209bdb76436dd.hdf5 was opened in r mode.
INFO:root:The file generator_data/segment_data_365d2f598e3e984cfd47499ae15209bdb76436dd.hdf5 was opened in r mode.


In [27]:
 multi_onsource_strain_plot = gf.generate_strain_plot(
        {"Onsource Noise" : multi_onsource[0]},
        title = f"Onsource Background noise at {multi_gps_times[0]}"
    )
    
multi_offsource_strain_plot = gf.generate_strain_plot(
        {"Offsource Noise" : multi_offsource[0]},
        title = f"Offsource Background noise at {multi_gps_times[0]}"
    )

multi_layout : List = [[multi_onsource_strain_plot, multi_offsource_strain_plot]]

# Arrange the plots in a grid. 
multi_grid = gridplot(multi_layout)
output_notebook()
show(multi_grid)

Loading BokehJS ...